# ProtRepr 快速入门指南

欢迎使用 ProtRepr！这个 notebook 将指导您快速上手使用我们的蛋白质表示学习框架。

## 简介

ProtRepr 是一个基于 `ProteinTensor` 的二次开发项目，专注于蛋白质表示学习、结构预测和功能分析。我们提供三种核心的蛋白质三维结构表示方法：

- **Atom14**: 紧凑型原子表示（14个关键原子）
- **Atom37**: 固定大小重原子表示（37个原子槽位） 
- **Frame**: 刚体坐标系表示（SE(3)变换）

所有计算都基于 PyTorch 后端，确保与深度学习工作流的无缝集成。


In [ ]:
# 导入必要的库
import torch
import numpy as np
from pathlib import Path

# 导入 ProtRepr 的核心数据类
# 注意：这些导入在实际实现后才能使用
try:
    from protrepr import Atom14, Atom37, Frame
    from protein_tensor import load_structure
    
    PROTREPR_AVAILABLE = True
    print("✅ ProtRepr 已成功导入")
except ImportError as e:
    PROTREPR_AVAILABLE = False
    print(f"⚠️  ProtRepr 尚未实现: {e}")
    print("本 notebook 将展示预期的使用方式")

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️  使用设备: {device}")

# 设置随机种子确保可重现性
torch.manual_seed(42)
np.random.seed(42)


## 1. 加载蛋白质结构

首先，我们需要加载一个蛋白质结构文件（PDB格式）并确保使用 PyTorch 后端。


In [ ]:
# 示例：加载蛋白质结构
# 注意：您需要提供实际的 PDB 文件路径

if PROTREPR_AVAILABLE:
    # 实际使用方式（需要真实的PDB文件）
    # pdb_file = "path/to/your/protein.pdb"
    # protein_tensor = load_structure(pdb_file)
    # 
    # # 验证使用了 PyTorch 后端
    # assert isinstance(protein_tensor.coordinates, torch.Tensor)
    # print(f"✅ 蛋白质结构已加载，坐标形状: {protein_tensor.coordinates.shape}")
    
    print("📝 请提供 PDB 文件路径来加载真实的蛋白质结构")
else:
    print("🔄 模拟数据演示:")
    
    # 创建模拟数据来演示预期的工作流程
    class MockProteinTensor:
        def __init__(self):
            # 模拟一个包含50个原子的小蛋白质
            self.coordinates = torch.randn(50, 3) * 10.0  # 原子坐标
            self.num_atoms = 50
            self.num_residues = 10  # 假设10个残基
            
        def to_torch(self):
            return {"coordinates": self.coordinates}
    
    mock_protein = MockProteinTensor()
    print(f"📊 模拟蛋白质: {mock_protein.num_atoms} 个原子, {mock_protein.num_residues} 个残基")
    print(f"📐 坐标形状: {mock_protein.coordinates.shape}")
    print(f"💾 设备: {mock_protein.coordinates.device}")


## 2. 创建三种蛋白质表示

现在我们将演示如何创建三种不同的蛋白质表示方法。每种表示都有其特定的用途和优势：

### 🧬 Atom14 表示
- **用途**: 紧凑型原子表示，节省内存
- **特点**: 每个残基14个关键原子位置
- **适用**: AlphaFold 输入特征工程、快速计算

### 🔬 Atom37 表示  
- **用途**: 完整的重原子表示
- **特点**: 每个残基37个原子槽位，涵盖所有可能的重原子
- **适用**: AlphaFold 主干网络、精确结构分析

### 🎯 Frame 表示
- **用途**: SE(3)-equivariant 网络的理想输入
- **特点**: 刚体坐标系（平移+旋转矩阵）
- **适用**: 几何深度学习、结构预测、分子动力学


In [ ]:
# 演示三种表示方法的创建
print("🚀 创建三种蛋白质表示方法...")

if PROTREPR_AVAILABLE:
    # 实际使用方式
    print("\n=== 真实使用方式 ===")
    
    # # 方式一：创建 Atom14 表示
    # atom14 = Atom14.from_protein_tensor(protein_tensor)
    # print(f"Atom14 坐标形状: {atom14.coords.shape}")
    # print(f"Atom14 掩码形状: {atom14.mask.shape}")
    # print(f"残基数量: {atom14.num_residues}")
    # 
    # # 方式二：创建 Atom37 表示
    # atom37 = Atom37.from_protein_tensor(protein_tensor)
    # print(f"Atom37 坐标形状: {atom37.coords.shape}")
    # print(f"Atom37 掩码形状: {atom37.mask.shape}")
    # 
    # # 方式三：创建 Frame 表示
    # frame = Frame.from_protein_tensor(protein_tensor)
    # print(f"Frame 平移形状: {frame.translations.shape}")
    # print(f"Frame 旋转形状: {frame.rotations.shape}")
    
    print("📝 请先加载真实的蛋白质结构")
    
else:
    print("\n=== 模拟演示（展示预期的API设计）===")
    
    # 模拟 Atom14 数据
    num_residues = 10
    print(f"\n🧬 Atom14 表示（{num_residues} 个残基）:")
    mock_atom14_coords = torch.randn(num_residues, 14, 3) * 10.0
    mock_atom14_mask = torch.rand(num_residues, 14) > 0.3  # 70% 原子存在
    print(f"  📐 坐标形状: {mock_atom14_coords.shape}")
    print(f"  🎭 掩码形状: {mock_atom14_mask.shape}")
    print(f"  📊 真实原子数: {mock_atom14_mask.sum().item():.0f}")
    
    # 模拟 Atom37 数据  
    print(f"\n🔬 Atom37 表示（{num_residues} 个残基）:")
    mock_atom37_coords = torch.randn(num_residues, 37, 3) * 10.0
    mock_atom37_mask = torch.rand(num_residues, 37) > 0.4  # 60% 原子存在
    print(f"  📐 坐标形状: {mock_atom37_coords.shape}")
    print(f"  🎭 掩码形状: {mock_atom37_mask.shape}")
    print(f"  📊 真实原子数: {mock_atom37_mask.sum().item():.0f}")
    
    # 模拟 Frame 数据
    print(f"\n🎯 Frame 表示（{num_residues} 个残基）:")
    mock_translations = torch.randn(num_residues, 3) * 10.0
    # 生成有效的旋转矩阵
    random_matrices = torch.randn(num_residues, 3, 3)
    Q, R = torch.linalg.qr(random_matrices)
    mock_rotations = Q
    print(f"  📍 平移向量形状: {mock_translations.shape}")
    print(f"  🔄 旋转矩阵形状: {mock_rotations.shape}")
    
    # 验证旋转矩阵的有效性
    det = torch.det(mock_rotations)
    print(f"  ✅ 旋转矩阵行列式均值: {det.mean():.4f} (应接近1.0)")
    
    print(f"\n💾 内存使用比较:")
    atom14_memory = mock_atom14_coords.numel() + mock_atom14_mask.numel()
    atom37_memory = mock_atom37_coords.numel() + mock_atom37_mask.numel()
    print(f"  Atom14: {atom14_memory:,} 个元素")
    print(f"  Atom37: {atom37_memory:,} 个元素") 
    print(f"  内存比率: {atom37_memory/atom14_memory:.2f}x")


## 3. 设备管理和深度学习集成

ProtRepr 设计时充分考虑了深度学习工作流的需求，支持无缝的设备管理和模型集成。


In [ ]:
# 设备管理演示
print("🖥️ 设备管理功能演示...")

if PROTREPR_AVAILABLE:
    print("\n=== 设备管理真实用法 ===")
    
    # # 设备间移动
    # atom14_cpu = atom14.to_device(torch.device("cpu"))
    # atom37_gpu = atom37.to_device(device)  # 移动到 GPU（如果可用）
    # frame_device = frame.to_device(device)
    # 
    # print(f"Atom14 设备: {atom14_cpu.device}")
    # print(f"Atom37 设备: {atom37_gpu.device}")
    # print(f"Frame 设备: {frame_device.device}")
    
    print("📝 请先创建数据类实例")
    
else:
    print("\n=== 设备管理模拟 ===")
    
    # 模拟设备管理
    print(f"当前设备: {device}")
    
    # 模拟数据在不同设备上的移动
    if device.type == "cuda":
        print("🔄 GPU 设备管理演示:")
        gpu_coords = mock_atom37_coords.to(device)
        cpu_coords = gpu_coords.cpu()
        print(f"  原始设备: {mock_atom37_coords.device}")
        print(f"  GPU 设备: {gpu_coords.device}")
        print(f"  CPU 设备: {cpu_coords.device}")
    else:
        print("💻 CPU 设备演示:")
        print(f"  当前在 CPU 上运行: {mock_atom37_coords.device}")

print("\n🤖 深度学习集成示例:")
print("""
# 典型的深度学习工作流
class ProteinClassifier(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.atom_embedding = torch.nn.Linear(37 * 3, 512)
        self.classifier = torch.nn.Linear(512, num_classes)
    
    def forward(self, atom37):
        # 直接使用 atom37.coords 和 atom37.mask
        coords = atom37.coords.flatten(start_dim=-2)
        features = self.atom_embedding(coords)
        return self.classifier(features.mean(dim=1))

# 使用示例
# model = ProteinClassifier(num_classes=10)
# logits = model(atom37)  # 直接传入数据类实例
""")

# 简单的张量操作演示
print(f"\n⚡ 张量操作演示:")
sample_coords = torch.randn(5, 37, 3)
sample_mask = torch.rand(5, 37) > 0.5

# 模拟一些常见的几何计算
distances = torch.norm(sample_coords, dim=-1)  # 计算到原点的距离
masked_distances = distances * sample_mask.float()  # 应用掩码

print(f"  样本形状: {sample_coords.shape}")
print(f"  距离统计: 均值={masked_distances.mean():.2f}, 标准差={masked_distances.std():.2f}")
print(f"  有效原子比例: {sample_mask.float().mean():.2%}")


## 🎉 总结与下一步

恭喜！您已经了解了 ProtRepr 框架的核心概念和基本用法。

### 📋 您学到了什么

1. **三种核心表示方法**: Atom14、Atom37、Frame
2. **数据类设计**: 使用 `@dataclass` 装饰器，支持属性访问
3. **设备管理**: 无缝的 CPU/GPU 转换
4. **深度学习集成**: 直接与 PyTorch 模型集成

### 🚀 下一步建议

1. **安装依赖**: 
   ```bash
   # 克隆项目
   git clone <repository-url>
   cd ProtRepr
   
   # 安装依赖（需要 Git SSH 访问权限）
   uv pip install -e .
   ```

2. **获取测试数据**: 下载一些 PDB 文件进行实验

3. **查看文档**: 
   - 📚 `docs/usage.md` - 详细使用指南
   - 🧪 `tests/` - 测试用例和示例
   - 📜 `scripts/example_usage.py` - 完整示例脚本

4. **探索高级功能**:
   - SE(3)-equivariant 网络集成
   - 批量处理和优化
   - 自定义几何计算

### 🔗 有用资源

- [ProteinTensor 库](https://github.com/a-green-hand-jack/ProteinTensor)
- [PyTorch 文档](https://pytorch.org/docs/)
- [AlphaFold 论文](https://www.nature.com/articles/s41586-021-03819-2)

### 💡 提示

- 始终确保使用 `torch` 后端来加载 ProteinTensor
- 利用 GPU 加速进行大规模计算
- 使用适合您任务的表示方法（内存 vs 精度权衡）

开始您的蛋白质表示学习之旅吧！🧬✨
